In [1]:
import numpy as np
import math
import nest
import nest.raster_plot
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib import rcParams 
import os.path


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.3
 Built: May  5 2022 07:35:57

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
# Variables
tones = [20]
angles = [0]
max_ild = 0
time_sim = 1000 #us

# Cell parameters
C_m_bushy = 1 #pF
V_m_bushy = -63.8 #mV

In [3]:
# Fixed
w_head = 22 #cm
v_sound = 33000 #cm/s

ANFs2SBCs_weight = 2.0
ANFs2GBCs_weight = 1.0
GBCs2MNTBCs_weight = 16.0

SBCs2LSO_weight = 16.0
MNTBCs2LSO_weight = -2.0

SBCs2MSO_weight = 4.0
MNTBCs2MSO_weights = [0, -2.0, -4.0, -6.0, -8.0, -10.0, -12.0, -14.0, -16.0, -20.0]
n_battery = len(MNTBCs2MSO_weights)

delay_inh = 0.1

mean_rate = 0
mean_amplitude = 1000
noise_rate = 0

n_ANFs = 35000
ANFs2SBCs = 4
ANFs2GBCs = 20
SBCs2MSOs = int(ANFs2GBCs/ANFs2SBCs)
SBCs2LSOs = int(ANFs2GBCs/ANFs2SBCs)
n_SBCs = int(n_ANFs/ANFs2SBCs)
n_GBCs = int(n_ANFs/ANFs2GBCs)
n_MSOs = n_GBCs*n_battery

In [ ]:
for tone in tones:
    
    results_r_MSO = np.zeros((n_battery, len(angles)))
    results_l_MSO = np.zeros((n_battery, len(angles)))

    results_r_SBC = np.zeros(len(angles))
    results_l_SBC = np.zeros(len(angles))

    results_r_GBC = np.zeros(len(angles))
    results_l_GBC = np.zeros(len(angles))

    for angle in angles:

        nest.ResetKernel()
        nest.local_num_threads = 16
        nest.resolution = 0.01 # 10 us = minimum audible angle (MAA) increment of 1.25°

        freq = np.round(np.logspace(np.log(20),np.log(20000), num = 3500, base = np.exp(1)),2)
        channel_x = np.where(freq>=tone)[0][0]
        spectro = np.zeros((3500,time_sim))
        amplitudes = np.round(stats.norm.pdf(np.linspace(-1, 1, 21) , 0, 1.0/(math.sqrt(2*math.pi)*1)),2)

        if(channel_x<10):
            spectro[channel_x:channel_x+10+1,:] = amplitudes[10:].reshape(11,1)*np.ones((11, time_sim))
            spectro[0:channel_x+1, :] = amplitudes[10-channel_x:11].reshape(channel_x+1,1)*np.ones((channel_x+1, time_sim))
        else:
            if(channel_x>3489):
                spectro[channel_x-10:channel_x+1] = amplitudes[:11].reshape(11,1)*np.ones((11, time_sim))
                spectro[channel_x:] = amplitudes[10:10+3500-channel_x].reshape(3500-channel_x,1)*np.ones((3500-channel_x, time_sim))
            else:
                spectro[channel_x - 10 : channel_x + 10 + 1, :] = amplitudes.reshape(21,1)*np.ones((21, time_sim))

        delta_x = (w_head*np.sin(np.deg2rad(angle)))
        itd = 1000*delta_x/v_sound #ms
        ipds = 2*np.pi*itd*freq/1000
        ild = angle*max_ild/90
        
        # Populations

        r_ANFs_amp = nest.Create('sinusoidal_poisson_generator',n_ANFs,
                params={'frequency': np.repeat(freq, 10),
                            'phase': np.repeat(np.rad2deg(ipds),10)}) #ITDs

        r_ANFs = nest.Create('parrot_neuron', n_ANFs)

        r_SBCs = nest.Create('iaf_cond_alpha', n_SBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': V_m_bushy})

        r_GBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': V_m_bushy})

        r_MNTBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': V_m_bushy})

        r_MSO = nest.Create('iaf_cond_alpha', n_MSOs, 
                         params = {'C_m': C_m_bushy, 'V_m': V_m_bushy})


        l_ANFs_amp = nest.Create('sinusoidal_poisson_generator',n_ANFs,
                    params={'frequency': np.repeat(freq, 10),
                        'phase': 0})

        l_ANFs = nest.Create('parrot_neuron',n_ANFs)

        l_SBCs = nest.Create('iaf_cond_alpha', n_SBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': V_m_bushy})

        l_GBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': V_m_bushy})

        l_MNTBCs = nest.Create('iaf_cond_alpha', n_GBCs, 
                         params = {'C_m': C_m_bushy, 'V_m': V_m_bushy})

        l_MSO = nest.Create('iaf_cond_alpha', n_MSOs, 
                         params = {'C_m': C_m_bushy, 'V_m': V_m_bushy})

        ANFs_noise = nest.Create('poisson_generator',1,
                         params = {'rate':noise_rate})

        s_rec_r = nest.Create('spike_recorder')
        s_rec_l = nest.Create('spike_recorder')
        
        # Connections

        nest.Connect(r_ANFs_amp, r_ANFs, 'one_to_one')
        nest.Connect(l_ANFs_amp, l_ANFs, 'one_to_one')

        nest.Connect(r_ANFs, s_rec_r, 'all_to_all')
        nest.Connect(l_ANFs, s_rec_l, 'all_to_all')

        nest.Connect(r_SBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_SBCs, s_rec_l, 'all_to_all')

        nest.Connect(r_GBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_GBCs, s_rec_l, 'all_to_all')

        nest.Connect(r_MNTBCs, s_rec_r, 'all_to_all')
        nest.Connect(l_MNTBCs, s_rec_l, 'all_to_all')

        nest.Connect(r_MSO, s_rec_r, 'all_to_all')
        nest.Connect(l_MSO, s_rec_l, 'all_to_all')


        for i in range(len(spectro)-1):
            if(np.any(spectro[i][:] > 0)):
                nest.Connect(ANFs_noise, r_ANFs[10*i:10*(i+1)], 'all_to_all')
                nest.Connect(ANFs_noise, l_ANFs[10*i:10*(i+1)], 'all_to_all')

        for i in range(n_SBCs):
            nest.Connect(r_ANFs[ANFs2SBCs*i:ANFs2SBCs*(i+1)], r_SBCs[i], 'all_to_all', syn_spec = {"weight":ANFs2SBCs_weight})
            nest.Connect(l_ANFs[ANFs2SBCs*i:ANFs2SBCs*(i+1)], l_SBCs[i], 'all_to_all', syn_spec = {"weight":ANFs2SBCs_weight})

        for i in range(n_GBCs):
            nest.Connect(r_ANFs[ANFs2GBCs*i:ANFs2GBCs*(i+1)], r_GBCs[i], 'all_to_all', syn_spec = {"weight":ANFs2GBCs_weight})
            nest.Connect(l_ANFs[ANFs2GBCs*i:ANFs2GBCs*(i+1)], l_GBCs[i], 'all_to_all', syn_spec = {"weight":ANFs2GBCs_weight})

        nest.Connect(r_GBCs, r_MNTBCs, 'one_to_one', syn_spec = {"weight":GBCs2MNTBCs_weight})
        nest.Connect(l_GBCs, l_MNTBCs, 'one_to_one', syn_spec = {"weight":GBCs2MNTBCs_weight})

        #MSO
        for i in range(n_GBCs):
            for j in range(n_battery):
                nest.Connect(r_SBCs[SBCs2MSOs*i:SBCs2MSOs*(i+1)], r_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight})
                nest.Connect(l_SBCs[SBCs2MSOs*i:SBCs2MSOs*(i+1)], r_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight}) 
                nest.Connect(r_SBCs[SBCs2MSOs*i:SBCs2MSOs*(i+1)], l_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight})
                nest.Connect(l_SBCs[SBCs2MSOs*i:SBCs2MSOs*(i+1)], l_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":SBCs2MSO_weight}) 

        for i in range(n_GBCs):
            for j in range(n_battery):
                nest.Connect(l_MNTBCs[i], r_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":MNTBCs2MSO_weights[j], "delay": delay_inh})
                nest.Connect(r_MNTBCs[i], l_MSO[i*n_battery+j], 'all_to_all', syn_spec = {"weight":MNTBCs2MSO_weights[j], "delay": delay_inh})


        mm = nest.Create('multimeter', 1, {'record_from': ['V_m']})
        nest.SetStatus(mm, {'interval': 0.01})
        nest.Connect(mm, r_MSO,'all_to_all')

        if(angle>=0): 
            mean_amplitude_r = mean_amplitude
            mean_amplitude_l = mean_amplitude/(10**(abs(ild)/20))
        else:
            mean_amplitude_l = mean_amplitude
            mean_amplitude_r = mean_amplitude/(10**(abs(ild)/20))

        def input_set_up(spectro, ms): 
            for r in range(0, len(spectro)-1):
                if spectro[r][ms] > 0:
                    r_ANFs_amp[10*r:10*(r+1)].set(amplitude = spectro[r][ms]*mean_amplitude_r)
                    l_ANFs_amp[10*r:10*(r+1)].set(amplitude = spectro[r][ms]*mean_amplitude_l)

        for i in range(time_sim):
            input_set_up(spectro,i)
            nest.Simulate(1)

        data_r = s_rec_r.get('events')
        data_l = s_rec_l.get('events')
        data_mso = mm.get('events')

        id_r_ANF1 = r_ANFs[0].get('global_id')
        id_r_SBC1 = r_SBCs[0].get('global_id')
        id_r_GBC1 = r_GBCs[0].get('global_id')
        id_r_MNTBC1 = r_MNTBCs[0].get('global_id')
        id_r_MSO1 = r_MSO[0].get('global_id')

        id_l_ANF1 = l_ANFs[0].get('global_id')
        id_l_SBC1 = l_SBCs[0].get('global_id')
        id_l_GBC1 = l_GBCs[0].get('global_id')
        id_l_MNTBC1 = l_MNTBCs[0].get('global_id')
        id_l_MSO1 = l_MSO[0].get('global_id')
        
        
        # Active Cells
        ac_r_anf = np.unique(data_r['senders'][np.where((data_r['senders']<id_r_SBC1) & (data_r['senders']>=id_r_ANF1))])
        ac_l_anf = np.unique(data_l['senders'][np.where((data_l['senders']<id_l_SBC1) & (data_l['senders']>=id_l_ANF1))])

        ac_r_sbc = np.unique(data_r['senders'][np.where((data_r['senders']>=id_r_SBC1) & (data_r['senders']<id_r_GBC1))])
        ac_l_sbc = np.unique(data_l['senders'][np.where((data_l['senders']>=id_l_SBC1) & (data_l['senders']<id_l_GBC1))])

        ac_r_gbc = np.unique(data_r['senders'][np.where((data_r['senders']>=id_r_GBC1) & (data_r['senders']<id_r_MNTBC1))])
        ac_l_gbc = np.unique(data_l['senders'][np.where((data_l['senders']>=id_l_GBC1) & (data_l['senders']<id_l_MNTBC1))])

        ac_r_mntbc = np.unique(data_r['senders'][np.where((data_r['senders']>=id_r_MNTBC1) & (data_r['senders']<id_r_MSO1))])
        ac_l_mntbc = np.unique(data_l['senders'][np.where((data_l['senders']>=id_l_MNTBC1) & (data_l['senders']<id_l_MSO1))])
        
        
        # Global Rates
        rate_r_anf = len(data_r['times'][np.where((data_r['senders']<id_r_SBC1) & (data_r['senders']>=id_r_ANF1))])*1000/time_sim
        rate_l_anf = len(data_l['times'][np.where((data_l['senders']<id_l_SBC1) & (data_l['senders']>=id_l_ANF1))])*1000/time_sim

        rate_r_sbc = len(data_r['times'][np.where((data_r['senders']>=id_r_SBC1) & (data_r['senders']<id_r_GBC1))])*1000/time_sim
        rate_l_sbc = len(data_l['times'][np.where((data_l['senders']>=id_l_SBC1) & (data_l['senders']<id_l_GBC1))])*1000/time_sim

        rate_r_gbc = len(data_r['times'][np.where((data_r['senders']>=id_r_GBC1) & (data_r['senders']<id_r_MNTBC1))])*1000/time_sim
        rate_l_gbc = len(data_l['times'][np.where((data_l['senders']>=id_l_GBC1) & (data_l['senders']<id_l_MNTBC1))])*1000/time_sim
        
        rate_r_mntbc = len(data_r['times'][np.where((data_r['senders']>=id_r_MNTBC1) & (data_r['senders']<id_r_MSO1))])*1000/time_sim
        rate_l_mntbc = len(data_l['times'][np.where((data_l['senders']>=id_l_MNTBC1) & (data_l['senders']<id_l_MSO1))])*1000/time_sim

        #MSO

        #averaging on total number of active batteries --> result: battery with rates for each inh weight
        rate_r_MSO = np.zeros(n_battery)
        rate_l_MSO = np.zeros(n_battery)

        for i in range(int(n_MSOs/n_battery)): #n of batteries
            for j in range(n_battery): # neurons for battery
                if(id_r_MSO1+n_battery*i+j in data_r['senders']):
                    rate_r_MSO[j] += (np.unique(data_r['senders'][np.where(data_r['senders'] == id_r_MSO1+n_battery*i+j)], return_counts= True)[1][0])
                else:
                    rate_r_MSO[j] += 0
                if(id_l_MSO1+n_battery*i+j in data_l['senders']):
                    rate_l_MSO[j] += (np.unique(data_l['senders'][np.where(data_l['senders'] == id_l_MSO1+n_battery*i+j)], return_counts= True)[1][0])
                else:
                    rate_l_MSO[j] += 0     


        results_r_SBC[np.where(angles == angle)[0][0]] = rate_r_sbc
        results_l_SBC[np.where(angles == angle)[0][0]] = rate_l_sbc

        results_r_GBC[np.where(angles == angle)[0][0]] = rate_r_gbc
        results_l_GBC[np.where(angles == angle)[0][0]] = rate_l_gbc

        results_r_MSO[:, np.where(angles == angle)[0][0]] = rate_r_MSO
        results_l_MSO[:, np.where(angles == angle)[0][0]] = rate_l_MSO

    """    
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results/SBC_r_freq_{}'.format(tone), results_r_SBC)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results/SBC_l_freq_{}'.format(tone), results_l_SBC)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results/GBC_r_freq_{}'.format(tone), results_r_GBC)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results/GBC_l_freq_{}'.format(tone), results_l_GBC)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results/MSO_r_freq_{}'.format(tone), results_r_MSO)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results/MSO_l_freq_{}'.format(tone), results_l_MSO)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results/LSO_r_freq_{}'.format(tone), results_r_LSO)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results/LSO_l_freq_{}'.format(tone), results_l_LSO)"""

    """np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results_noILD/SBC_r_freq_{}'.format(tone), results_r_SBC)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results_noILD/SBC_l_freq_{}'.format(tone), results_l_SBC)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results_noILD/GBC_r_freq_{}'.format(tone), results_r_GBC)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results_noILD/GBC_l_freq_{}'.format(tone), results_l_GBC)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results_noILD/MSO_r_freq_{}'.format(tone), results_r_MSO)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results_noILD/MSO_l_freq_{}'.format(tone), results_l_MSO)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results_noILD/LSO_r_freq_{}'.format(tone), results_r_LSO)
    np.savetxt('/home/ubuntu/cosyne-tutorial-2022/Results_noILD/LSO_l_freq_{}'.format(tone), results_l_LSO)"""


Sep 20 09:45:41 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.01 ms.


In [ ]:
ac_r_mso

In [ ]:
id_r_MSO1

In [ ]:
np.unique(data_r['senders'][np.where(data_r['senders']>=id_r_MSO1)],  return_counts= True)

In [ ]:
colors = ['c', 'deepskyblue', 'royalblue']
inhs = [0,1,3]
fig, ax = plt.subplots(1, figsize=(8,7), sharey = True)

from matplotlib import rcParams 
text_color = 'k' 
rcParams['text.color'] = text_color 
rcParams['axes.labelcolor'] = text_color 
rcParams['xtick.color'] = text_color 
rcParams['ytick.color'] = text_color 
plt.rc('font', size=16)          # controls default text sizes 
plt.rc('axes', titlesize=20)     # fontsize of the axes title 
plt.rc('axes', labelsize=18)    # fontsize of the x and y labels 
plt.rc('xtick', labelsize=18)    # fontsize of the tick labels 
plt.rc('ytick', labelsize=18)    # fontsize of the tick labels 
plt.rc('legend', fontsize=16)    # legend fontsize 
plt.rc('figure', titlesize=26)  # fontsize of the figure title
ax.spines['bottom'].set_color('white')
ax.spines['left'].set_color('white')
ax.xaxis.label.set_color('white')
ax.tick_params(axis='x', colors='white')
ax.yaxis.label.set_color('white')
ax.tick_params(axis='y', colors='white')


ax.axhline(y = r_MSO[0].get('V_th'), color = 'r', linestyle='--', label = "Threshold")
ax.plot(data_mso['times'][np.where(data_mso['senders'] == 86321  + 9)], data_mso['V_m'][np.where(data_mso['senders'] == 86321  + 9)], color = 'w', label = 'No Inhibition')
ax.axhline(y = r_MSO[0].get('V_th'), color = 'r', linestyle='--')
                          
ax.plot(data_mso['times'][np.where(data_mso['senders'] == 86321)], data_mso['V_m'][np.where(data_mso['senders'] == 86321)], color = 'deepskyblue', label = "Inhibition Weight = {} nS".format(MNTBCs2MSO_weights[0]))
ax.plot(data_mso['times'][np.where(data_mso['senders'] == 86324)], data_mso['V_m'][np.where(data_mso['senders'] == 86324)], color = 'royalblue', label = "Inhibition Weight = {} nS".format(MNTBCs2MSO_weights[3]))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlim(4,10)
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Vm [mV]")
fig.legend()
plt.savefig('demo.png', transparent=True)

In [ ]:
colors = ['c', 'deepskyblue', 'royalblue']
inhs = [0,1,3]
fig, ax = plt.subplots(1,3, figsize=(24,7), sharey = True)
for i in range(3):
    inh = inhs[i]

    from matplotlib import rcParams 
    text_color = 'black' 
    rcParams['text.color'] = text_color 
    rcParams['axes.labelcolor'] = text_color 
    rcParams['xtick.color'] = text_color 
    rcParams['ytick.color'] = text_color 
    plt.rc('font', size=16)          # controls default text sizes 
    plt.rc('axes', titlesize=20)     # fontsize of the axes title 
    plt.rc('axes', labelsize=18)    # fontsize of the x and y labels 
    plt.rc('xtick', labelsize=18)    # fontsize of the tick labels 
    plt.rc('ytick', labelsize=18)    # fontsize of the tick labels 
    plt.rc('legend', fontsize=16)    # legend fontsize 
    plt.rc('figure', titlesize=26)  # fontsize of the figure title
    
    
    if(i==0): 
        ax[i].axhline(y = r_MSO[0].get('V_th'), color = 'r', linestyle='--', label = "Threshold")
        ax[i].plot(data_mso['times'][np.where(data_mso['senders'] == 86321  + i)], data_mso['V_m'][np.where(data_mso['senders'] == 86321  + 9)], color = 'k', label = 'No Inhibition')
        
    else:
        ax[i].plot(data_mso['times'][np.where(data_mso['senders'] == 86321  + i)], data_mso['V_m'][np.where(data_mso['senders'] == 86321  + 9)], color = 'k')
        ax[i].axhline(y = r_MSO[0].get('V_th'), color = 'r', linestyle='--')
    ax[i].plot(data_mso['times'][np.where(data_mso['senders'] == 86321  + i)], data_mso['V_m'][np.where(data_mso['senders'] == 86321 +inh)], color = colors[i], label = "Inhibition Weight = {} nS".format(MNTBCs2MSO_weights[inh]))
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].set_xlim(4,10)
    ax[i].set_xlabel("Time [ms]")
    ax[i].set_ylabel("Vm [mV]")

    fig.legend()

In [ ]:
fig, ax = plt.subplots(6,1, figsize=(24,16))
colors = ['c', 'deepskyblue', 'dodgerblue', 'royalblue', 'steelblue']
for i in range(5):
    ax[0].plot(data_mso['times'][np.where(data_mso['senders'] == 86321  + i)], data_mso['V_m'][np.where(data_mso['senders'] == 86321  + i)], color = colors[i])
    ax[0].set_xticks(np.arange(0,100,10))
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].xaxis.label.set_color('white')
    ax[0].tick_params(axis='x', colors='white')
    ax[0].yaxis.label.set_color('white')
    ax[0].tick_params(axis='y', colors='white')
    ax[0].spines['bottom'].set_color('white')
    ax[0].spines['left'].set_color('white')
ax[0].set_ylabel("Vm [mV]")
#ax[0].set_ylabel("All", rotation = 1, fontsize=20, labelpad=20)
#ax[0].set_ylabel("Vm [mV]")
ax[0].set_yticklabels([])


for i in range(1,6):
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)

    #ax[i].set_title("Inhibition = {}".format(MNTBCs2MSO_weights[i-1]))
    ax[i].plot(data_mso['times'][np.where(data_mso['senders'] == 86321 + i)], data_mso['V_m'][np.where(data_mso['senders'] == 86321  + i)], color = colors[i-1])
    ax[i].axhline(y = r_MSO[0].get('V_th'), color = 'r', linestyle='--', label = "Threshold")
    ax[i].plot(data_r['times'][np.where(data_r['senders'] == 86321 + i)],  -55*np.ones(len(data_r['times'][np.where(data_r['senders'] == 86321 + i)])), color = 'w', marker = '*', ms = 14, linestyle = 'None', label = "Spikes")
    #ax[i].set_ylabel("{} nS".format(MNTBCs2MSO_weights[i-1]), rotation = 1, fontsize=20, labelpad=20)
    ax[i].set_ylabel("Vm [mV]")
    #ax[i].set_ylabel("Vm [mV]")
    ax[i].set_xticks(np.arange(0,100,10))
    ax[i].set_yticklabels([])
    ax[i].spines['bottom'].set_color('white')
    ax[i].spines['left'].set_color('white')
    ax[i].xaxis.label.set_color('white')
    ax[i].tick_params(axis='x', colors='white')
    ax[i].yaxis.label.set_color('white')
    ax[i].tick_params(axis='y', colors='white')
    plt.savefig('demo2.png', transparent=True)
ax[5].set_xlabel("Time [ms]")
handles, labels = ax[5].get_legend_handles_labels()
fig.legend(handles, labels, loc=2)
plt.savefig('demo2.png', transparent=True)

In [ ]:
id_r_ANF1 = r_ANFs[0].get('global_id')
id_r_SBC1 = r_SBCs[0].get('global_id')
id_r_GBC1 = r_GBCs[0].get('global_id')
id_r_MNTBC1 = r_MNTBCs[0].get('global_id')
id_r_MSO1 = r_MSO[0].get('global_id')
print(id_r_ANF1,id_r_SBC1,id_r_GBC1,id_r_MNTBC1, id_r_MSO1)
id_l_ANF1 = l_ANFs[0].get('global_id')
id_l_SBC1 = l_SBCs[0].get('global_id')
id_l_GBC1 = l_GBCs[0].get('global_id')
id_l_MNTBC1 = l_MNTBCs[0].get('global_id')
print(id_l_ANF1,id_l_SBC1,id_l_GBC1,id_l_MNTBC1)

In [ ]:
sim_time = 100 #ms

plt.figure(figsize=(15, 1))
plt.plot(data_r['times'][np.where((data_r['senders'] >= id_r_SBC1) & (data_r['senders'] < id_r_GBC1))], 
         data_r['senders'][np.where((data_r['senders'] >= id_r_SBC1) & (data_r['senders'] < id_r_GBC1))], ".", color = 'g')
plt.plot(data_l['times'][np.where((data_l['senders'] >= id_l_SBC1) & (data_l['senders'] < id_l_GBC1))], 
         data_l['senders'][np.where((data_l['senders'] >= id_l_SBC1) & (data_l['senders'] < id_l_GBC1))]-99750, ".", color = 'm')
plt.xlim(0,sim_time)
plt.title("Right vs Left SBCs spikes")
plt.ylabel("ID")
plt.xlabel("Time [ms]")
plt.xticks(ticks = np.arange(sim_time))
plt.grid()


plt.figure(figsize=(15, 1))
plt.plot(data_l['times'][np.where((data_l['senders'] >= id_l_MNTBC1))], 
         data_l['senders'][np.where((data_l['senders'] >= id_l_MNTBC1))] - 99750, ".", color = 'm')
plt.xlim(0,sim_time)
plt.title("Left MNTBCs spikes")
plt.ylabel("ID")
plt.xlabel("Time [ms]")
plt.xticks(ticks = np.arange(sim_time))
plt.grid()

plt.figure(figsize=(15, 1))
plt.plot(data_r['times'][np.where(data_r['senders'] >= id_r_MSO1)], 
         data_r['senders'][np.where(data_r['senders'] >= id_r_MSO1)], ".", color = 'g')
plt.xlim(0,sim_time)
plt.title("Right vs Left LSO spikes")
plt.ylabel("ID")
plt.xlabel("Time [ms]")
plt.xticks(ticks = np.arange(sim_time))
plt.grid()

In [ ]:
sim_time = 100 #ms

plt.figure(figsize=(15, 1))
plt.plot(data_r['times'][np.where(data_r['senders'] < id_r_SBC1)], 
         data_r['senders'][np.where(data_r['senders'] < id_r_SBC1)], ".", color = 'g')
plt.plot(data_l['times'][np.where(data_l['senders'] < id_l_SBC1)], 
         data_l['senders'][np.where(data_l['senders'] < id_l_SBC1)]- 99750, ".", color = 'm')
plt.xlim(0,sim_time)
plt.title("Right vs Left ANFs spikes")
plt.ylabel("ID")
plt.xlabel("Time [ms]")
plt.xticks(ticks = np.arange(sim_time))
plt.grid()

plt.figure(figsize=(15, 1))
plt.plot(data_r['times'][np.where((data_r['senders'] >= id_r_SBC1) & (data_r['senders'] < id_r_GBC1))], 
         data_r['senders'][np.where((data_r['senders'] >= id_r_SBC1) & (data_r['senders'] < id_r_GBC1))], ".", color = 'g')
plt.plot(data_l['times'][np.where((data_l['senders'] >= id_l_SBC1) & (data_l['senders'] < id_l_GBC1))], 
         data_l['senders'][np.where((data_l['senders'] >= id_l_SBC1) & (data_l['senders'] < id_l_GBC1))]-99750, ".", color = 'm')
plt.xlim(0,sim_time)
plt.title("Right vs Left SBCs spikes")
plt.ylabel("ID")
plt.xlabel("Time [ms]")
plt.xticks(ticks = np.arange(sim_time))
plt.grid()

plt.figure(figsize=(15, 1))
plt.plot(data_r['times'][np.where((data_r['senders'] >= id_r_GBC1)& (data_r['senders'] < id_r_MNTBC1))], 
         data_r['senders'][np.where((data_r['senders'] >= id_r_GBC1)& (data_r['senders'] < id_r_MNTBC1))], ".", color = 'g')
plt.plot(data_l['times'][np.where((data_l['senders'] >= id_l_GBC1)& (data_l['senders'] < id_l_MNTBC1))], 
         data_l['senders'][np.where((data_l['senders'] >= id_l_GBC1)& (data_l['senders'] < id_l_MNTBC1))] - 99750, ".", color = 'm')
plt.xlim(0,sim_time)
plt.title("Right vs Left GBCs spikes")
plt.ylabel("ID")
plt.xlabel("Time [ms]")
plt.xticks(ticks = np.arange(sim_time))
plt.grid()

plt.figure(figsize=(15, 1))
plt.plot(data_r['times'][np.where((data_r['senders'] >= id_r_MNTBC1)& (data_r['senders'] < id_r_MSO1))], 
         data_r['senders'][np.where((data_r['senders'] >= id_r_MNTBC1)& (data_r['senders'] < id_r_MSO1))], ".", color = 'g')
plt.plot(data_l['times'][np.where((data_l['senders'] >= id_l_MNTBC1))], 
         data_l['senders'][np.where((data_l['senders'] >= id_l_MNTBC1))] - 99750, ".", color = 'm')
plt.xlim(0,sim_time)
plt.title("Right vs Left MNTBCs spikes")
plt.ylabel("ID")
plt.xlabel("Time [ms]")
plt.xticks(ticks = np.arange(sim_time))
plt.grid()

plt.figure(figsize=(15, 1))
plt.plot(data_r['times'][np.where(data_r['senders'] >= id_r_MSO1)], 
         data_r['senders'][np.where(data_r['senders'] >= id_r_MSO1)], ".", color = 'g')
plt.xlim(0,sim_time)
plt.title("Right vs Left LSO spikes")
plt.ylabel("ID")
plt.xlabel("Time [ms]")
plt.xticks(ticks = np.arange(sim_time))
plt.grid()

In [ ]:
lim = 100
plt.figure(figsize=(20,28))
plt.subplot(911)
for i in range(8):
    plt.plot(data_mso['times'][np.where(data_mso['senders'] ==  92141 + i)], data_mso['V_m'][np.where(data_mso['senders'] ==  92141 + i)])
    plt.xlim(0,lim)
    plt.grid()
    
for i in range(8):
    plt.subplot(9,1, i+2)
    plt.title("Inhibition = {}".format(MNTBCs2MSO_weights[i]))
    plt.plot(data_mso['times'][np.where(data_mso['senders'] ==  92141 + i)], data_mso['V_m'][np.where(data_mso['senders'] ==  92141 + i)])
    plt.xlim(0,lim)
    plt.grid()